In [9]:
import pandas as pd
import shutil
import os
from tqdm.notebook import tqdm

In [10]:
PATH = "/datalake/ufmg/c04/tmp/288_licitacoes-pirapetinga/data/files/"
# list all files and remove file_description
files = os.listdir(PATH)
files.remove("file_description.jsonl")

# Read file_description
file_description = pd.read_json(os.path.join(PATH, "file_description.jsonl"), lines=True)
file_description.head()

,url,crawler_id,instance_id,crawled_at_date,referer,wait_crawler_finish_to_download,time_between_downloads,file_name,type,extracted_files
0,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 21:58:50.994079,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,bd9cf430f53cfa701f8c7d51e2e0ba2f,application/pdf,[]
1,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 21:58:50.994526,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,bc0866e315473780ab8d83b14278998f,application/pdf,[]
2,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:00:40.249297,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,c7bd8928fa0ff4856199e1eb2589e8dc,application/pdf,[]
3,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:00:40.249461,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,25bff9c51ff33ab73b3ec2a5c1ae8ea7,application/pdf,[]
4,https://pirapetinga.mg.gov.br/transparencia/do...,3,160616852568494,2020-11-23 22:00:40.248959,https://pirapetinga.mg.gov.br/transparencia/in...,False,1,5df4600b8c2d6bfc9fbdb9034a937966,application/pdf,[]


In [11]:
# Group and count types
file_description.groupby('type').size()

type
application/msword                                                            8
application/pdf                                                            1007
application/vnd.ms-excel                                                      9
application/vnd.openxmlformats-officedocument.spreadsheetml.sheet             3
application/vnd.openxmlformats-officedocument.wordprocessingml.document      81
dtype: int64

In [12]:
# Mapping extensions to their MIME types
# https://stackoverflow.com/questions/4212861/what-is-a-correct-mime-type-for-docx-pptx-etc
mime_dict = {     
    "application/pdf" : ".pdf", 
    "application/msword" : ".doc",
    #"application/msword" : ".dot",
    "application/vnd.openxmlformats-officedocument.wordprocessingml.document" : ".docx",
    "application/vnd.openxmlformats-officedocument.wordprocessingml.template" : ".dotx",
    "application/vnd.ms-word.document.macroEnabled.12" : ".docm",
    "application/vnd.ms-word.template.macroEnabled.12" : ".dotm",
    "application/vnd.ms-excel" : ".xls",
    #"application/vnd.ms-excel" : ".xlt",
    #"application/vnd.ms-excel" : ".xla",
    "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet" : ".xlsx",
    "application/vnd.openxmlformats-officedocument.spreadsheetml.template" : ".xltx",
    "application/vnd.ms-excel.sheet.macroEnabled.12" : ".xlsm",
    "application/vnd.ms-excel.template.macroEnabled.12" : ".xltm",
    "application/vnd.ms-excel.addin.macroEnabled.12" : ".xlam",
    "application/vnd.ms-excel.sheet.binary.macroEnabled.12" : ".xlsb",
    "application/vnd.ms-powerpoint" : ".ppt",
    #"application/vnd.ms-powerpoint" : ".pot",
    #"application/vnd.ms-powerpoint" : ".pps",
    #"application/vnd.ms-powerpoint" : ".ppa",
    "application/vnd.openxmlformats-officedocument.presentationml.presentation" : ".pptx",
    "application/vnd.openxmlformats-officedocument.presentationml.template" : ".potx",
    "application/vnd.openxmlformats-officedocument.presentationml.slideshow" : ".ppsx",
    "application/vnd.ms-powerpoint.addin.macroEnabled.12" : ".ppam",
    "application/vnd.ms-powerpoint.presentation.macroEnabled.12" : ".pptm",
    "application/vnd.ms-powerpoint.template.macroEnabled.12" : ".potm",
    "application/vnd.ms-powerpoint.slideshow.macroEnabled.12" : ".ppsm",
    "application/vnd.ms-access" : ".mdb"
}

# Rename files adding their extension
for file in tqdm(files):
    file = file.split(".")[0]
    # get file extension
    file_type = file_description.loc[file_description['file_name'] == file, "type"].values[0]
    file_extension = mime_dict[file_type]
    # set source and destination
    src = os.path.join(PATH, file)    
    dst = ".".join([src, file_extension])
    # rename file
    #shutil.move(src, dst)
    # update file_description
    file_description.loc[file_description['file_name'] == file, "file_name"] = "".join([file, file_extension])

# Write file
file_description.to_json(os.path.join(PATH, "file_description.jsonl"), lines=True, orient='records')